# LambdaRank by XGBoost model

In [53]:
import xgboost as xgb

from sklearn.datasets import load_svmlight_file

### Transform data to group data

In [ ]:
def save_data(group_data,output_feature,output_group):
    if len(group_data) == 0:
        return

    output_group.write(str(len(group_data))+"\n")
    for data in group_data:
        # only include nonzero features
        feats = [ p for p in data[2:] if float(p.split(':')[1]) != 0.0 ]        
        output_feature.write(data[0] + " " + " ".join(feats) + "\n")

In [ ]:
fi = open("./data/MSLR-WEB10K/Fold1/train.txt")
output_feature = open("./data/MSLR-WEB10K/Fold1/train_features.group","w")
output_group = open("./data/MSLR-WEB10K/Fold1/train.group","w")

group_data = []
group = ""
for line in fi:
    if not line:
        break
    if "#" in line:
        line = line[:line.index("#")]
    splits = line.strip().split(" ")
    if splits[1] != group:
        save_data(group_data,output_feature,output_group)
        group_data = []
    group = splits[1]
    group_data.append(splits)

save_data(group_data,output_feature,output_group)

fi.close()
output_feature.close()
output_group.close()

In [ ]:
fi = open("./data/MSLR-WEB10K/Fold1/vali.txt")
output_feature = open("./data/MSLR-WEB10K/Fold1/vali_features.group","w")
output_group = open("./data/MSLR-WEB10K/Fold1/vali.group","w")

group_data = []
group = ""
for line in fi:
    if not line:
        break
    if "#" in line:
        line = line[:line.index("#")]
    splits = line.strip().split(" ")
    if splits[1] != group:
        save_data(group_data,output_feature,output_group)
        group_data = []
    group = splits[1]
    group_data.append(splits)

save_data(group_data,output_feature,output_group)

fi.close()
output_feature.close()
output_group.close()

### Generate dataset

In [58]:
x_train, y_train = load_svmlight_file("./data/MSLR-WEB10K/Fold1/train.txt")
x_valid, y_valid = load_svmlight_file("./data/MSLR-WEB10K/Fold1/vali.txt")

In [65]:
train_dmatrix = xgb.DMatrix(x_train, y_train)
valid_dmatrix = xgb.DMatrix(x_valid, y_valid)

In [66]:
group_train = []
with open("./data/MSLR-WEB10K/Fold1/train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

In [68]:
group_valid = []
with open("./data/MSLR-WEB10K/Fold1/vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

In [69]:
train_dmatrix.set_group(group_train)

In [70]:
valid_dmatrix.set_group(group_valid)

In [73]:
params = {'objective': 'rank:pairwise', 
          'eta': 0.1, 
          'gamma': 1.0,
          'min_child_weight': 0.1, 
          'max_depth': 6,
          'eval_metric': 'ndcg@5'}
xgb_model = xgb.train(params, train_dmatrix, num_boost_round=4,
                           evals=[(valid_dmatrix, 'validation')])

[13:59:38] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[0]	validation-ndcg@5:0.389448
[13:59:47] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[1]	validation-ndcg@5:0.393292
[13:59:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[2]	validation-ndcg@5:0.400664
[14:00:06] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[3]	validation-ndcg@5:0.405078
